<a href="https://colab.research.google.com/github/hcorbel/attention_rank_bert_stackoverflow/blob/main/Stackoverflow_attention_rank_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/attention_rank/P7')

In [ ]:
import os
import time
import nltk.data

In [ ]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
"""
preprocessing document text to avoid breaking sentences and misunderstanding period
"""

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
start_time = time.time()
path_list = ['/content/gdrive/MyDrive/attention_rank/P7/stackoverflow']

In [ ]:
for item in path_list:
    dataset_path = item + '/'
    text_path = dataset_path + 'docsutf8/'
    output_path = dataset_path + 'processed_' + item + '/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    files = os.listdir(text_path)
    for i, file in enumerate(files):
        files[i] = file[:-4]

    files = files[:]
    ## instead of one example files = files[:1]

    for n, file in enumerate(files):

        fp = open(text_path + file + '.txt', errors="ignore")
        # ai ajouté errors="ignore"
        text = []
        for a,line in enumerate(fp):
            line = line.replace('\n','')
            if line:
                text.append(line)
        text = ''.join(text)
        #print(text)

        # replace or remove something for better sentences splitting
        text = text.replace('      ', '. ').replace('     ', '. ').replace('   ', ' ')\
            .replace('..', '.').replace(',.', ',').replace(':.', ':').replace('?.', ':')
        text = text.replace('Fig.', 'Figure').replace('Fig .', 'Figure')\
            .replace('FIG.', 'Figure').replace('FIG .', 'Figure').replace('et al.', '').replace('e.g.', '')

        # split full document text into sentences
        sentences = tokenizer.tokenize(text)
        #print(sentences)

        # double security for wrongly break a sentence into 2 pieces
        # if the heading character of a piece is lowercase, combine it to the last piece
        lower_lines = []
        for l, sentence in enumerate(sentences):
            if sentence[0].islower():
                if sentences[l-1][-1] == '.':
                    sentences[l - 1] = sentences[l-1][:-1]
                sentences[l-1] += ' ' + sentence
                lower_lines.append(l)
        #print('empty means good', lower_lines)

        final_output = []
        for l, sentence in enumerate(sentences):
            # if the heading character of a piece is lowercase, combine it to the last piece
            if l not in lower_lines and any(c.isalpha() for c in sentence):  # number_only pieces are dropped, like: '1.', '3.1.'
                final_output.append(sentence)
        #print(final_output)

        save_path = dataset_path + 'processed_docsutf8/'
        if not os.path.exists(save_path):
            os.mkdir(save_path)

        text_file = open(save_path+file+".txt", "w")
        text_file.write('$$$$$$'.join(final_output))
        text_file.close()

        print(n + 1, "th file", file, "running time", time.time() - start_time)

1 th file file_12 running time 12.074652910232544
2 th file file_84 running time 12.76214075088501
3 th file file_19 running time 12.806665897369385
4 th file file_97 running time 13.769248485565186
5 th file file_96 running time 13.775821208953857
6 th file file_25 running time 13.782207250595093
7 th file file_58 running time 13.78989839553833
8 th file file_67 running time 13.798341035842896
9 th file file_43 running time 13.803743839263916
10 th file file_81 running time 13.809549570083618
11 th file file_89 running time 13.815261840820312
12 th file file_28 running time 13.820624351501465
13 th file file_56 running time 13.825842380523682
14 th file file_24 running time 13.83076286315918
15 th file file_47 running time 13.836674213409424
16 th file file_8 running time 13.860742568969727
17 th file file_38 running time 13.867972373962402
18 th file file_87 running time 13.873560190200806
19 th file file_85 running time 13.879451274871826
20 th file file_61 running time 13.885680437

In [ ]:
import os
import argparse
import random
import time
from bert import tokenization

In [ ]:
"""
split document text into paired sentences, get ready to tokenize them feed bert self-attention extractor"""


def prep_document(document, max_sequence_length):
    """Does BERT-style preprocessing on the provided document."""
    max_num_tokens = max_sequence_length - 3
    target_seq_length = max_num_tokens

    # We DON"T just concatenate all of the tokens from a document into a long
    # sequence and choose an arbitrary split point because this would make the
    # next sentence prediction task too easy. Instead, we split the input into
    # segments "A" and "B" based on the actual "sentences" provided by the user
    # input.
    instances = []
    current_chunk = []
    current_length = 0
    i = 0
    while i < len(document):
        segment = document[i]
        current_chunk.append(segment)
        current_length += len(segment)
        if i == len(document) - 1 or current_length >= target_seq_length:
            if current_chunk:
                a_end = 1
                if len(current_chunk) >= 2:
                    a_end = random.randint(1, len(current_chunk) - 1)

                tokens_a = []
                for j in range(a_end):
                    tokens_a.extend(current_chunk[j])

                tokens_b = []
                for j in range(a_end, len(current_chunk)):
                    tokens_b.extend(current_chunk[j])
                truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, random)

                if len(tokens_a) == 0 or len(tokens_b) == 0:
                    break
                assert len(tokens_a) >= 1
                assert len(tokens_b) >= 1

                tokens = []
                tokens.append("[CLS]")
                for token in tokens_a:
                    tokens.append(token)
                tokens.append("[SEP]")

                for token in tokens_b:
                    tokens.append(token)
                tokens.append("[SEP]")

                instances.append(tokens)

            current_chunk = []
            current_length = 0
        i += 1
    return instances


def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng):
    """Truncates a pair of sequences to a maximum sequence length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if rng.random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()




In [ ]:
# main
dataset = '/content/gdrive/MyDrive/attention_rank/P7/stackoverflow'
text_path = dataset + '/processed_docsutf8/'
output_path = dataset + '/processed_stackoverflow' + '/'
save_path = output_path + 'sentence_paired_text/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

bert_dir = "/content/gdrive/MyDrive/attention_rank/P7/pretrained_bert/orgbert/"

files = os.listdir(text_path)
for i, file in enumerate(files):
    files[i] = file[:-4]

files = files[:]

start_time = time.time()

super_long = 0

parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument("--num-docs", default=1000, type=int, help="Number of documents to use (default=1000).")
parser.add_argument("--cased", default=False, action='store_true', help="Don't lowercase the input.")
args, unknown = parser.parse_known_args()
tokenizer = tokenization.FullTokenizer(vocab_file=os.path.join(bert_dir, "vocab.txt"), do_lower_case=not args.cased)

In [ ]:
for n, file in enumerate(files):
    text = ''
    my_file = text_path + file + '.txt'
    with open(my_file, "r") as f:
        for line in f:
            if line:
                text += line

    file1 = open(save_path + file + "_sentence_paired.txt", "a")

    for L in text.split('$$$$$$'):

        sub_write = 0
        line = tokenization.convert_to_unicode(L).strip()
        tokens = tokenizer.tokenize(line)
        current_doc_tokens = [tokens, tokens]

        """
        Check if the sentence will beyond max token length or not,
        if beyond, separate sentence to finer."""

        for segment in prep_document(current_doc_tokens, 512):
            # if the sentence length beyond max token number, break sentence by ';' and ','
            if (len(segment) - 1) / 2 != segment.index("[SEP]"):
                # print(segment)
                sub_write = 1
                super_long += 1
                cline = L.replace(",", ",$$$$$").replace(";", ";$$$$$").split("$$$$$")
                for part in cline:
                    # print(part)
                    file1.writelines(part)
                    file1.writelines("\n")
                    file1.writelines(part)
                    file1.writelines("\n")
                    file1.writelines("\n\n")
        # good to write
        if sub_write != 1:
            file1.writelines(L)
            file1.writelines("\n")
            file1.writelines(L)
            file1.writelines("\n")
            file1.writelines("\n\n")

    file1.close()

    run_time = time.time()
    print(n, "th file", file, "running time", run_time - start_time)

print('number of super long sentences:', super_long)

0 th file file_4 running time 8.473639488220215
1 th file file_0 running time 9.074121475219727
2 th file file_1 running time 9.819212436676025
3 th file file_2 running time 10.388495445251465
4 th file file_3 running time 10.95952558517456
5 th file file_12 running time 10.969563722610474
6 th file file_84 running time 10.978993654251099
7 th file file_19 running time 10.988776206970215
8 th file file_97 running time 10.994969844818115
9 th file file_96 running time 11.000992059707642
10 th file file_25 running time 11.008186101913452
11 th file file_58 running time 11.016031503677368
12 th file file_67 running time 11.02213978767395
13 th file file_43 running time 11.030181884765625
14 th file file_81 running time 11.057009220123291
15 th file file_89 running time 11.065306425094604
16 th file file_28 running time 11.074278831481934
17 th file file_56 running time 11.08293104171753
18 th file file_24 running time 11.08949613571167
19 th file file_47 running time 11.096256494522095
20

In [ ]:
"""Does BERT-style preprocessing of unlabeled data; heavily based on
create_pretraining_data.py in the BERT codebase. However, does not mask words
or ever use random paragraphs for the second text segment."""

import argparse
import os
import random
import time
import utils
from bert import tokenization

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
"""
tokenize paired sentences, get ready to feed bert self-attention extractor"""


def prep_document(document, max_sequence_length):
    """Does BERT-style preprocessing on the provided document."""
    max_num_tokens = max_sequence_length - 3
    target_seq_length = max_num_tokens

    # We DON"T just concatenate all of the tokens from a document into a long
    # sequence and choose an arbitrary split point because this would make the
    # next sentence prediction task too easy. Instead, we split the input into
    # segments "A" and "B" based on the actual "sentences" provided by the user
    # input.
    instances = []
    current_chunk = []
    current_length = 0
    i = 0
    while i < len(document):
        segment = document[i]
        # print(len(document), i, 'stop 2', segment)
        current_chunk.append(segment)
        current_length += len(segment)
        if i == len(document) - 1 or current_length >= target_seq_length:
            if current_chunk:
                # print("cc", current_chunk, len(current_chunk))
                a_end = 1
                if len(current_chunk) >= 2:
                    a_end = random.randint(1, len(current_chunk) - 1)

                tokens_a = []
                for j in range(a_end):
                    tokens_a.extend(current_chunk[j])
                # print("aa", tokens_a)

                tokens_b = []
                # print("lc", len(current_chunk))
                for j in range(a_end, len(current_chunk)):
                    # print("j=", j)
                    tokens_b.extend(current_chunk[j])
                truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, random)
                # print("bb", tokens_b)

                if len(tokens_a) == 0 or len(tokens_b) == 0:
                    break
                assert len(tokens_a) >= 1
                assert len(tokens_b) >= 1

                tokens = []
                tokens.append("[CLS]")
                for token in tokens_a:
                    tokens.append(token)
                # print("cs", tokens)
                tokens.append("[SEP]")

                for token in tokens_b:
                    tokens.append(token)
                tokens.append("[SEP]")

                instances.append(tokens)

            current_chunk = []
            current_length = 0
        i += 1
    # print(instances)
    return instances


def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng):
    """Truncates a pair of sequences to a maximum sequence length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if rng.random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()



In [ ]:
dataset = '/content/gdrive/MyDrive/attention_rank/P7/stackoverflow'
text_path = dataset + '/processed_docsutf8/'
output_path = dataset + '/processed_stackoverflow' + '/'
save_path = output_path + 'sentence_paired_text/'

# set which pre-trained bert is going to use
bert_dir = "/content/gdrive/MyDrive/attention_rank/P7/pretrained_bert/orgbert/"
bert_name = bert_dir.split('/')[1]

files = os.listdir(text_path)
for i, file in enumerate(files):
    files[i] = file[:-4]

files = files[:]

start_time = time.time()
parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument("--num-docs", default=1000, type=int,help="Number of documents to use (default=1000).")
parser.add_argument("--cased", default=False, action='store_true',help="Don't lowercase the input.")
parser.add_argument("--max_sequence_length", default=512, type=int, help="Maximum input sequence length after tokenization (default=128).")
args, unknown = parser.parse_known_args()
tokenizer = tokenization.FullTokenizer(vocab_file=os.path.join(bert_dir, "vocab.txt"), do_lower_case=not args.cased)

for n, file in enumerate(files):
    data_file = save_path + file + '_sentence_paired.txt'
    random.seed(0)
    current_doc_tokens = []
    segments = []

    with open(data_file, "r") as f:
        raw_lines = []
        for l, line in enumerate(f):
            raw_lines.append(line)
            line = tokenization.convert_to_unicode(line).strip()
            # print(line)
            if not line:  # line is empty, deal the 2 segments in the [current doc tokens]
                if current_doc_tokens:

                    for segment in prep_document(
                            current_doc_tokens, args.max_sequence_length):
                        if (len(segment) - 1) / 2 != segment.index("[SEP]"):
                            # print("0", segment.index("[SEP]"), len(segment) - 1)
                            # print(l, segment)
                            cline = raw_lines[l - 1].replace("),", "),$$$$$").split("$$$$$")
                            # for part in cline:
                            # print(part)
                        # print(segment)
                        segments.append(segment)
                        if len(segments) >= args.num_docs:
                            break
                    if len(segments) >= args.num_docs:
                        break

                current_doc_tokens = []  # clean up [current doc tokens]

            tokens = tokenizer.tokenize(line)
            if tokens:  # if line is not empty, add lines to [current doc tokens]
                current_doc_tokens.append(tokens)
    run_time = time.time()
    print(n, "th file", file, "running time", run_time - start_time)
    utils.write_json([{"tokens": s} for s in segments],
                     data_file.replace("_sentence_paired.txt", "")+"_"+bert_name+".json")

0 th file file_4 running time 0.08666753768920898
1 th file file_0 running time 0.8966028690338135
2 th file file_1 running time 1.497527837753296
3 th file file_2 running time 2.0931649208068848
4 th file file_3 running time 2.6513452529907227
5 th file file_12 running time 3.2333762645721436
6 th file file_84 running time 3.2437708377838135
7 th file file_19 running time 3.2592251300811768
8 th file file_97 running time 3.267986536026001
9 th file file_96 running time 3.2749838829040527
10 th file file_25 running time 3.283597230911255
11 th file file_58 running time 3.292379140853882
12 th file file_67 running time 3.299969434738159
13 th file file_43 running time 3.309337615966797
14 th file file_81 running time 3.31650972366333
15 th file file_89 running time 3.325962781906128
16 th file file_28 running time 3.337966203689575
17 th file file_56 running time 3.3474698066711426
18 th file file_24 running time 3.353924512863159
19 th file file_47 running time 3.3777732849121094
20 th